In [78]:
# All the imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Input, Conv1D, Flatten, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [79]:
# -----------------------------
# STEP 1: Load the dataset
# -----------------------------
df = pd.read_csv('AirQuality.csv')
df.head(20)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
5,3/10/2004,23:00:00,1.2,1197,38,4.7,750,89,1337,96,1393,949,11.2,59.2,0.7848
6,3/11/2004,0:00:00,1.2,1185,31,3.6,690,62,1462,77,1333,733,11.3,56.8,0.7603
7,3/11/2004,1:00:00,1.0,1136,31,3.3,672,62,1453,76,1333,730,10.7,60.0,0.7702
8,3/11/2004,2:00:00,0.9,1094,24,2.3,609,45,1579,60,1276,620,10.7,59.7,0.7648
9,3/11/2004,3:00:00,0.6,1010,19,1.7,561,-200,1705,-200,1235,501,10.3,60.2,0.7517


In [80]:
# -----------------------------
# STEP 2: Create DateTime index
# -----------------------------
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
DateTime,,,,,,,,,,,,,,,
2004-03-10 18:00:00,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
2004-03-10 19:00:00,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2004-03-10 20:00:00,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
2004-03-10 21:00:00,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
2004-03-10 22:00:00,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-04-04 10:00:00,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
2005-04-04 11:00:00,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
2005-04-04 12:00:00,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406


In [81]:
# -----------------------------
# STEP 3: Keep only CO(GT)
# -----------------------------
co_data = df[['CO(GT)']]
co_data

,CO(GT)
DateTime,
2004-03-10 18:00:00,2.6
2004-03-10 19:00:00,2.0
2004-03-10 20:00:00,2.2
2004-03-10 21:00:00,2.2
2004-03-10 22:00:00,1.6
...,...
2005-04-04 10:00:00,3.1
2005-04-04 11:00:00,2.4
2005-04-04 12:00:00,2.4


In [82]:
# Replace -200 (sensor missing values) with NaN
co_data = co_data.replace(-200, np.nan)
co_data

,CO(GT)
DateTime,
2004-03-10 18:00:00,2.6
2004-03-10 19:00:00,2.0
2004-03-10 20:00:00,2.2
2004-03-10 21:00:00,2.2
2004-03-10 22:00:00,1.6
...,...
2005-04-04 10:00:00,3.1
2005-04-04 11:00:00,2.4
2005-04-04 12:00:00,2.4


In [83]:

# -----------------------------
# STEP 4: Convert hourly → 3-hour average
# -----------------------------
co_3h = co_data.resample('3h').mean()
co_3h

,CO(GT)
DateTime,
2004-03-10 18:00:00,2.266667
2004-03-10 21:00:00,1.666667
2004-03-11 00:00:00,1.033333
2004-03-11 03:00:00,0.650000
2004-03-11 06:00:00,1.266667
...,...
2005-04-04 00:00:00,0.666667
2005-04-04 03:00:00,0.450000
2005-04-04 06:00:00,3.366667


In [84]:
# -----------------------------
# STEP 5: Fill missing values
# -----------------------------
co_3h = co_3h.interpolate()
co_3h

,CO(GT)
DateTime,
2004-03-10 18:00:00,2.266667
2004-03-10 21:00:00,1.666667
2004-03-11 00:00:00,1.033333
2004-03-11 03:00:00,0.650000
2004-03-11 06:00:00,1.266667
...,...
2005-04-04 00:00:00,0.666667
2005-04-04 03:00:00,0.450000
2005-04-04 06:00:00,3.366667


In [85]:
# -----------------------------
# STEP 6: Normalize (0–1)
# -----------------------------
scaler = MinMaxScaler()
co_scaled = scaler.fit_transform(co_3h)
co_scaled

array([[0.1951952 ],
       [0.14114114],
       [0.08408408],
       ...,
       [0.29429429],
       [0.27327327],
       [0.19219219]], shape=(3119, 1))

In [86]:
# -----------------------------
# STEP 7: Create sequences
# -----------------------------
def make_dataset(data, steps):
    X, y = [], []
    for i in range(len(data) - steps):
        X.append(data[i:i+steps])
        y.append(data[i+steps])
    return np.array(X), np.array(y)
    
# Past 24 hours → predict next value
n_steps = 8   # 8 × 3 hours = 24 hours
X, y = make_dataset(co_scaled, n_steps)

# -----------------------------
# STEP 8: Train–test split
# -----------------------------
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Training input shape:", X_train.shape)

Training input shape: (2488, 8, 1)


# LSTM Model

In [46]:

lstm_base = Sequential([
    LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

lstm_base.compile(
    optimizer='adam',
    loss='mse'
)

lstm_base.summary()
lstm_base.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 32)                  │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0144 - val_loss: 0.0124
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0104 - val_loss: 0.0119
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0101 - val_loss: 0.0116
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0097 - val_loss: 0.0112
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0094 - val_loss: 0.0109
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0089 - val_loss: 0.0098
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0080 - val_loss: 0.0092
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0074 - val_loss: 0.0088
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0071 - val_loss: 0.0079
Epoch 10/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0067 - val_loss: 0.0076
Epoch 11/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0066 - val_loss: 0.0075
Epoch 12/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0067 - val_l

# GRU 

In [47]:
gru_base = Sequential([
    GRU(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

gru_base.compile(
    optimizer='adam',
    loss='mse'
)

gru_base.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0175 - val_loss: 0.0127
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0108 - val_loss: 0.0113
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0098 - val_loss: 0.0103
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0093 - val_loss: 0.0095
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0088 - val_loss: 0.0089
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0082 - val_loss: 0.0083
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0073 - val_loss: 0.0077
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0073 - val_loss: 0.0071
Epoch 10/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0069 - val_loss: 0.0071
Epoch 11/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0069 - val_loss: 0.0070
Epoch 12/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0068 - val_l

# TCN

In [48]:
tcn_base = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu',
           input_shape=(X_train.shape[1], X_train.shape[2])),
    Flatten(),
    Dense(1)
])

tcn_base.compile(
    optimizer='adam',
    loss='mse'
)

tcn_base.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/20


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0090 - val_loss: 0.0084
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0064 - val_loss: 0.0076
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0059 - val_loss: 0.0072
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0057 - val_loss: 0.0071
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0056 - val_loss: 0.0070
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0055 - val_loss: 0.0069
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0054 - val_loss: 0.0065
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0052 - val_loss: 0.0064
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0052 - val_loss: 0.0062
Epoch 10/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0062
Epoch 11/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0062
Epoch 12/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0060

In [51]:
def evaluate(model, name):
    y_pred = model.predict(X_test, verbose=0)

    y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
    y_pred_inv = scaler.inverse_transform(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)

    print(f"\n{name}")
    print("RMSE:", rmse)
    print("MAE :", mae)
    print("R²  :", r2)

evaluate(lstm_base, "Baseline LSTM")
evaluate(gru_base, "Baseline GRU")
evaluate(tcn_base, "Baseline TCN")



Baseline LSTM
RMSE: 0.9505004755762778
MAE : 0.7414008618421641
R²  : 0.4338168802320578

Baseline GRU
RMSE: 0.9433224870570697
MAE : 0.6990180174892379
R²  : 0.4423359932062799

Baseline TCN
RMSE: 0.8162706920292159
MAE : 0.5924452366613334
R²  : 0.582438309414491


# Model Improvement

In [68]:
def make_dataset(data, steps):
    X, y = [], []
    for i in range(len(data) - steps):
        X.append(data[i:i+steps])
        y.append(data[i+steps])
    return np.array(X), np.array(y)
    
# Past 48 hours → predict next value
n_steps = 16   # 16 × 3 hours = 48 hours history
X, y = make_dataset(co_scaled, n_steps)

# -----------------------------
# STEP 8: Train–test split
# -----------------------------
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Training input shape:", X_train.shape)

Training input shape: (2482, 16, 1)


In [69]:
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=15,               # Give it more room to "hiccup" and recover
    restore_best_weights=True  # This is your insurance policy
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

In [70]:
lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')

history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    callbacks=[early_stop],
    validation_split=0.1,
    verbose=1
)

Epoch 1/40


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0132 - val_loss: 0.0147
Epoch 2/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0113 - val_loss: 0.0129
Epoch 3/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0111 - val_loss: 0.0125
Epoch 4/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0112 - val_loss: 0.0121
Epoch 5/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0109 - val_loss: 0.0122
Epoch 6/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0108 - val_loss: 0.0148
Epoch 7/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0109 - val_loss: 0.0116
Epoch 8/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0103 - val_loss: 0.0109
Epoch 9/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0098 - val_loss: 0.0094
Epoch 10/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0086 - val_loss: 0.0079
Epoch 11/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0080 - val_loss: 0.0073
Epoch 12/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0078 - val_l

In [71]:
# GRU
gru_model = Sequential([
    GRU(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    GRU(32),
    Dense(1)
])

gru_model.compile(optimizer='adam', loss='mse')

history_gru = gru_model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - loss: 0.0122 - val_loss: 0.0109
Epoch 2/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0102 - val_loss: 0.0102
Epoch 3/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0091 - val_loss: 0.0081
Epoch 4/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0082 - val_loss: 0.0071
Epoch 5/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0079 - val_loss: 0.0070
Epoch 6/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0078 - val_loss: 0.0065
Epoch 7/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0076 - val_loss: 0.0061
Epoch 8/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0072 - val_loss: 0.0065
Epoch 9/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0071 - val_loss: 0.0061
Epoch 10/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0069 - val_loss: 0.0065
Epoch 11/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0068 - val_loss: 0.0059
Epoch 12/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0

In [72]:
# TCN

tcn_model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(1)
])

tcn_model.compile(optimizer='adam', loss='mse')

history_tcn = tcn_model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/40


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0118 - val_loss: 0.0105
Epoch 2/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0076 - val_loss: 0.0085
Epoch 3/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0067 - val_loss: 0.0067
Epoch 4/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0058 - val_loss: 0.0060
Epoch 5/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0054 - val_loss: 0.0053
Epoch 6/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0050 - val_loss: 0.0052
Epoch 7/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0048 - val_loss: 0.0052
Epoch 8/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0048 - val_loss: 0.0048
Epoch 9/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0046 - val_loss: 0.0047
Epoch 10/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0045 - val_loss: 0.0048
Epoch 11/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 12/40
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0043 - val_loss: 0.

In [73]:
def evaluate(model, name):
    y_pred = model.predict(X_test, verbose=0)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
    y_pred_inv = scaler.inverse_transform(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)

    print(f"\n{name} Performance:")
    print("RMSE:", rmse)
    print("MAE :", mae)
    print("R²  :", r2)
    return rmse, mae, r2

evaluate(lstm_model, "Refined LSTM")
evaluate(gru_model, "Refined GRU")
evaluate(tcn_model, "Refined TCN")



Refined LSTM Performance:
RMSE: 0.8862014892006931
MAE : 0.6597318347845421
R²  : 0.509280618864655

Refined GRU Performance:
RMSE: 1.1358846043984205
MAE : 0.8574374559484883
R²  : 0.19381133741217904

Refined TCN Performance:
RMSE: 0.7421682578169593
MAE : 0.5444235208609767
R²  : 0.6558300001661705


(np.float64(0.7421682578169593), 0.5444235208609767, 0.6558300001661705)

# More tuning

In [87]:
# Past 36 hours → predict next 3-hour CO
n_steps = 12  # 12 × 3h = 36 hours
X, y = make_dataset(co_scaled, n_steps)

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Training input shape:", X_train.shape)


Training input shape: (2485, 12, 1)


In [88]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-5,
    verbose=1
)


In [89]:

lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')

history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=16,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


Epoch 1/60


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - loss: 0.0125 - val_loss: 0.0151 - learning_rate: 0.0010
Epoch 2/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0117 - val_loss: 0.0133 - learning_rate: 0.0010
Epoch 3/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0112 - val_loss: 0.0122 - learning_rate: 0.0010
Epoch 4/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0110 - val_loss: 0.0130 - learning_rate: 0.0010
Epoch 5/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0107 - val_loss: 0.0131 - learning_rate: 0.0010
Epoch 6/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0106 - val_loss: 0.0116 - learning_rate: 0.0010
Epoch 7/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0104 - val_loss: 0.0107 - learning_rate: 0.0010
Epoch 8/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0097 - val_loss: 0.0102 - learning_rate: 0.0010
Epoch 9/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0088 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 10/60

In [93]:
# GRU model with more units and lower dropout
gru_model = Sequential([
    GRU(256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.15),
    GRU(128),
    Dense(1)
])

gru_model.compile(optimizer='adam', loss='mse')

# Train GRU
history_gru = gru_model.fit(
    X_train, y_train,
    epochs=80,          # slightly longer to allow convergence
    batch_size=8,       # smaller batch size
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


Epoch 1/80


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


280/280 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - loss: 0.0097 - val_loss: 0.0075 - learning_rate: 0.0010
Epoch 2/80
280/280 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0084 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 3/80
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0076
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - loss: 0.0077 - val_loss: 0.0064 - learning_rate: 0.0010
Epoch 4/80
280/280 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - loss: 0.0069 - val_loss: 0.0066 - learning_rate: 5.0000e-04
Epoch 5/80
280/280 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 0.0062 - val_loss: 0.0063 - learning_rate: 5.0000e-04
Epoch 6/80
279/280 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0059
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
280/280 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - loss: 0.0054 - val_loss: 0.0060 - learning_rate: 5.0000e-04
Epoch 7/80
280/280 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - loss: 

In [91]:
tcn_model = Sequential([
    Conv1D(128, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Conv1D(64, kernel_size=5, activation='relu'),
    Dropout(0.3),
    Conv1D(32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(1)
])

tcn_model.compile(optimizer='adam', loss='mse')

history_tcn = tcn_model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=16,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


C:\Users\ASUS1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 0.0095 - val_loss: 0.0107 - learning_rate: 0.0010
Epoch 2/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0067 - val_loss: 0.0068 - learning_rate: 0.0010
Epoch 3/60
139/140 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0062
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0061 - val_loss: 0.0062 - learning_rate: 0.0010
Epoch 4/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0053 - val_loss: 0.0059 - learning_rate: 5.0000e-04
Epoch 5/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0052 - val_loss: 0.0058 - learning_rate: 5.0000e-04
Epoch 6/60
136/140 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0050 
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0049 - val_loss: 0.0056 - learning_rate: 5.0000e-04
Epoch 7/60
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - l

In [92]:
def evaluate(model, name):
    y_pred = model.predict(X_test, verbose=0)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
    y_pred_inv = scaler.inverse_transform(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)

    print(f"\n{name} Performance:")
    print("RMSE:", rmse)
    print("MAE :", mae)
    print("R²  :", r2)
    return rmse, mae, r2

evaluate(lstm_model, "Refined LSTM")
evaluate(gru_model, "Refined GRU")
evaluate(tcn_model, "Refined TCN")



Refined LSTM Performance:
RMSE: 0.7865409986454904
MAE : 0.5767114606482636
R²  : 0.6129193553337703

Refined GRU Performance:
RMSE: 1.132458082431446
MAE : 0.862520897164861
R²  : 0.19757768707633216

Refined TCN Performance:
RMSE: 0.741655912510001
MAE : 0.5471976459473531
R²  : 0.6558374176129331


(np.float64(0.741655912510001), 0.5471976459473531, 0.6558374176129331)

In [31]:
# Further more improvement using Data Augumentattion
